# 🎯 Cebu Pacific Customer Support Agent Optimization

**Complete DSPy Agent Optimization with MLflow Tracking**

This notebook demonstrates automated AI agent optimization using:
- **DSPy Framework**: Prompt optimization
- **Groq LLM**: llama-3.1-8b-instant (fast inference)
- **MIPROv2 Optimizer**: Automatic instruction generation
- **MLflow**: Experiment tracking and logging

## Demonstration Flow:

### **STEP 1: Show the Problem**
  → Display real messy support ticket
  → Run unoptimized agent (generic, unhelpful response)
  → Metrics: Time = 5 min, Quality = 30%

### **STEP 2: Show DSPy Optimization**
  → Load 50 past tickets with good resolutions
  → Run MIPROv2 optimization (3-5 minutes)
  → Track with MLflow
  → Show optimized instructions + few-shot examples

### **STEP 3: Show the Results**
  → Re-run same ticket with optimized agent
  → Detailed, step-by-step solution
  → Metrics: Time = 30 sec, Quality = 85%

### **STEP 4: Show Business Impact**
  → Calculate: 1000 tickets/day × 4.5 min saved = 75 hours/day
  → 75 hours × $30/hour = $2,250/day = $821K/year savings
  → ROI: ♾️ (optimization costs ~$1)

**Technology Stack:**
- DSPy with MIPROv2
- Groq llama-3.1-8b-instant
- MLflow experiment tracking
- Custom evaluation metrics

**Dataset:**
- 50 training examples (successful resolutions)
- 20 validation examples (test scenarios)
- Real Cebu Pacific support tickets

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 2: Install and Import Required Packages
# ═══════════════════════════════════════════════════════════════════════════

# Install required packages (run once)
import sys

# Note: Run this in terminal first if packages not installed:
# pip install dspy-ai mlflow matplotlib numpy python-dotenv

try:
    import dspy
    import mlflow
    import matplotlib.pyplot as plt
    print("✓ All packages already installed!")
except ImportError:
    print("Installing required packages...")
    !{sys.executable} -m pip install -q dspy-ai mlflow matplotlib numpy python-dotenv
    import dspy
    import mlflow
    import matplotlib.pyplot as plt
    print("✓ Packages installed successfully!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 3: Import All Required Libraries
# ═══════════════════════════════════════════════════════════════════════════

import os
import json
import time
import numpy as np
from typing import List, Dict, Any
from datetime import datetime
from functools import lru_cache

print("✅ All libraries imported successfully!")
print(f"   DSPy version: {dspy.__version__ if hasattr(dspy, '__version__') else 'Unknown'}")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Python version: {sys.version.split()[0]}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 4: Helper Functions (Recommended Production Practice)
# ═══════════════════════════════════════════════════════════════════════════

def get_groq_api_key() -> str:
    """
    Retrieve Groq API key from environment with fallback options.

    Priority:
    1. Environment variable GROQ_API_KEY
    2. .env file
    3. Direct input (not recommended for production)

    Returns:
        str: Groq API key
    """
    # Try environment variable first
    api_key = os.getenv("GROQ_API_KEY")

    if api_key:
        return api_key

    # Try loading from .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
        api_key = os.getenv("GROQ_API_KEY")
        if api_key:
            return api_key
    except ImportError:
        pass

    # Fallback: Direct input (set your key here)
    # WARNING: For production, use environment variables or .env file
    api_key = "your_groq_api_key_here"  # ⚠️ REPLACE THIS

    if api_key == "your_groq_api_key_here":
        raise ValueError(
            "\n\n"
            "🚨 GROQ API KEY NOT CONFIGURED!\n"
            "\n"
            "Please set your Groq API key using ONE of these methods:\n"
            "\n"
            "Method 1 (Recommended): Environment Variable\n"
            "  export GROQ_API_KEY='your_key_here'\n"
            "\n"
            "Method 2: .env file\n"
            "  Create .env file with: GROQ_API_KEY=your_key_here\n"
            "\n"
            "Method 3: Direct in this cell (not recommended)\n"
            "  Replace 'your_groq_api_key_here' with actual key\n"
            "\n"
            "Get your FREE API key: https://console.groq.com/keys\n"
        )

    return api_key


def get_mlflow_tracking_uri() -> str:
    """
    Get MLflow tracking URI with fallback to local server.

    Returns:
        str: MLflow tracking URI
    """
    # Try environment variable first
    uri = os.getenv("MLFLOW_TRACKING_URI")

    if uri:
        return uri

    # Default: local MLflow server
    # Start server with: mlflow server --host 127.0.0.1 --port 8080
    return "http://localhost:8080"


def setup_mlflow_experiment(experiment_name: str = "cebu_pacific_optimization") -> str:
    """
    Configure MLflow experiment with autologging.

    Args:
        experiment_name: Name of MLflow experiment

    Returns:
        str: Experiment ID
    """
    # Set tracking URI
    tracking_uri = get_mlflow_tracking_uri()
    mlflow.set_tracking_uri(tracking_uri)

    # Set or create experiment
    try:
        experiment = mlflow.set_experiment(experiment_name)
        experiment_id = experiment.experiment_id
    except Exception as e:
        print(f"⚠️  MLflow experiment setup warning: {e}")
        print("   Continuing without MLflow tracking...")
        return None

    # Enable comprehensive autologging for DSPy
    try:
        mlflow.dspy.autolog(
            log_evals=True,              # Log evaluation results
            log_compiles=True,           # Log compilation process
            log_traces_from_compile=True # Log detailed traces during optimization
        )
        print(f"✅ MLflow autologging enabled")
    except Exception as e:
        print(f"⚠️  MLflow autologging warning: {e}")

    return experiment_id


print("✅ Helper functions defined:")
print("   - get_groq_api_key()")
print("   - get_mlflow_tracking_uri()")
print("   - setup_mlflow_experiment()")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 5: Setup Groq API Key
# ═══════════════════════════════════════════════════════════════════════════

# Load Groq API key
try:
    groq_api_key = get_groq_api_key()
    os.environ["GROQ_API_KEY"] = groq_api_key

    print("✅ Groq API key configured")
    print(f"   Key: {groq_api_key[:20]}... (hidden)")

except ValueError as e:
    print(str(e))
    raise

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 6: Configure MLflow Tracking
# ═══════════════════════════════════════════════════════════════════════════

# Setup MLflow experiment tracking
experiment_id = setup_mlflow_experiment("cebu_pacific_optimization")

if experiment_id:
    print("✅ MLflow tracking configured")
    print(f"   URI: {mlflow.get_tracking_uri()}")
    print(f"   Experiment: cebu_pacific_optimization")
    print(f"   Experiment ID: {experiment_id}")
    print("\n💡 View results:")
    print(f"   {mlflow.get_tracking_uri()}")
else:
    print("⚠️  MLflow tracking not available (continuing without it)")
    print("   To enable: Start MLflow server with:")
    print("   $ mlflow server --host 127.0.0.1 --port 8080")